In [1]:
import pandas as pd

import pyspark
from pyspark.sql.functions import *

In [2]:
# create spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/05/22 22:50:59 WARN Utils: Your hostname, Brandyns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.3.168 instead (on interface en0)
22/05/22 22:50:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 22:51:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1. Create a spark data frame that contains your favorite programming languages.
The name of the column should be language
View the schema of the dataframe
Output the shape of the dataframe
Show the first 5 records in the dataframe

In [3]:
# Create languages df in pandas
languages = pd.DataFrame({"language" : ['r', 'python', 'sql', 'java', 'html']})

In [4]:
# Create spark df
df = spark.createDataFrame(languages)

In [5]:
# View schema
df.printSchema()

root
 |-- language: string (nullable = true)



In [6]:
# View shape of df
print('shape: ', df.count(), " x ", len(df.columns))

shape:  5  x  1


In [7]:
# First five rows
df.head(5)

[Row(language='r'),
 Row(language='python'),
 Row(language='sql'),
 Row(language='java'),
 Row(language='html')]

2. Load the mpg dataset as a spark dataframe.
- Create 1 column of output that contains a message like:
The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.
- Transform the trans column so that it only contains either manual or auto.

In [8]:
import pydataset

mpg = spark.createDataFrame(pydataset.data('mpg'))

In [15]:
mpg.describe().show()

+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|             displ|             year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|               234|              234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null| 3.471794871794872|           2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.2919590310839348|4.509646313320436|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166448|null|   null|
|    min|        audi|      4runne

In [9]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [19]:
# Show output 'The 1999 audi a4 has a 4 clyinder engine.'
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a '), 
                  mpg.cyl, lit(' cylinder engine.'))).show(truncate = False)

+------------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine.)|
+------------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                                     |
|The 1999 audi a4 has a 4 cylinder engine.                                     |
|The 2008 audi a4 has a 4 cylinder engine.                                     |
|The 2008 audi a4 has a 4 cylinder engine.                                     |
|The 1999 audi a4 has a 6 cylinder engine.                                     |
|The 1999 audi a4 has a 6 cylinder engine.                                     |
|The 2008 audi a4 has a 6 cylinder engine.                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.                             |
|The 1999 audi a4 quattro has a 4 cylinder engine.                             |
|The 2008 audi a4 quattro ha

In [21]:
# Transform the 'trans' column so that it only contains either manual or auto
mpg.select('trans',
          regexp_extract('trans', r"^(\w+)\(", 1)).show()

+----------+----------------------------------+
|     trans|regexp_extract(trans, ^(\w+)\(, 1)|
+----------+----------------------------------+
|  auto(l5)|                              auto|
|manual(m5)|                            manual|
|manual(m6)|                            manual|
|  auto(av)|                              auto|
|  auto(l5)|                              auto|
|manual(m5)|                            manual|
|  auto(av)|                              auto|
|manual(m5)|                            manual|
|  auto(l5)|                              auto|
|manual(m6)|                            manual|
|  auto(s6)|                              auto|
|  auto(l5)|                              auto|
|manual(m5)|                            manual|
|  auto(s6)|                              auto|
|manual(m6)|                            manual|
|  auto(l5)|                              auto|
|  auto(s6)|                              auto|
|  auto(s6)|                            

In [23]:
# Regexp_replace and when
mpg.select('trans', 
          regexp_replace('trans', r"\(.+$", "").alias("regexp_replace"),
          when(mpg.trans.like("auto%"), 'auto').otherwise('manual').alias('when + like')).show()

+----------+--------------+-----------+
|     trans|regexp_replace|when + like|
+----------+--------------+-----------+
|  auto(l5)|          auto|       auto|
|manual(m5)|        manual|     manual|
|manual(m6)|        manual|     manual|
|  auto(av)|          auto|       auto|
|  auto(l5)|          auto|       auto|
|manual(m5)|        manual|     manual|
|  auto(av)|          auto|       auto|
|manual(m5)|        manual|     manual|
|  auto(l5)|          auto|       auto|
|manual(m6)|        manual|     manual|
|  auto(s6)|          auto|       auto|
|  auto(l5)|          auto|       auto|
|manual(m5)|        manual|     manual|
|  auto(s6)|          auto|       auto|
|manual(m6)|        manual|     manual|
|  auto(l5)|          auto|       auto|
|  auto(s6)|          auto|       auto|
|  auto(s6)|          auto|       auto|
|  auto(l4)|          auto|       auto|
|  auto(l4)|          auto|       auto|
+----------+--------------+-----------+
only showing top 20 rows



3. Load the tips dataset as a spark df

In [24]:
tips = spark.createDataFrame(pydataset.data('tips'))

In [25]:
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [26]:
# What percentage of observations are smokers?
tips.groupby('smoker').count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [27]:
tips.groupby('smoker').count().withColumn('percent', col('count') / tips.count()).show()

+------+-----+-------------------+
|smoker|count|            percent|
+------+-----+-------------------+
|    No|  151| 0.6188524590163934|
|   Yes|   93|0.38114754098360654|
+------+-----+-------------------+



In [28]:
tips.groupby('smoker').count().withColumn('percent', concat(round((col('count') / tips.count() * 100), 0)
                                                            .cast('int'), lit("%"))).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



In [54]:
tips.agg(mean(when(tips.smoker == 'Yes', 1).otherwise(0)).alias('Percent_smokers')).show()

+-------------------+
|    Percent_smokers|
+-------------------+
|0.38114754098360654|
+-------------------+



In [33]:
# Create a column that contains the tip percentage
tips.withColumn('tip_percentage', expr('tip / total_bill')).show()

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|0.18623962040332148|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|0.22805017103762829|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|0.11607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|0.13031914893617022|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| 0.2185385656292287|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 0.1665043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|0

In [35]:
# Calculate average tip percentage for each combination of sex and smoker
(
tips.withColumn('tip_percentage', tips.tip / tips.total_bill)
.groupby('sex')
.pivot('smoker')
.agg(round(mean('tip_percentage'), 4))
.show()
)

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



4. Seattle Weather dataset

In [44]:
from vega_datasets import data

weather = data.seattle_weather()
weather = spark.createDataFrame(weather)

In [45]:
weather.show()

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10 00:00:00|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11 00:00:00|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12 00:00:00|          0.0|     6.1|    -1.7| 1.9|    

In [46]:
# Convert temperatures from c to f

weather = weather.withColumn("temp_max", (col('temp_max') * 9 / 5 + 32)).withColumn("temp_min", (weather.temp_min * 9 / 5 + 32))    

weather.show()

+-------------------+-------------+------------------+------------------+----+-------+
|               date|precipitation|          temp_max|          temp_min|wind|weather|
+-------------------+-------------+------------------+------------------+----+-------+
|2012-01-01 00:00:00|          0.0|             55.04|              41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|             48.02|             37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|        

In [47]:
# Which month has the most rain on average
row = (
weather.withColumn('month', month('date'))
.withColumn('year', year('date'))
.groupby('month', 'year')
.agg(sum('precipitation').alias('total_monthly_prec'))
.groupby('month')
.agg(mean('total_monthly_prec').alias('avg_monthly_rain'))
.sort(col('avg_monthly_rain').desc())
.first()
)

In [48]:
row

Row(month=11, avg_monthly_rain=160.625)

In [50]:
(
weather.withColumn('month', month('date'))
.groupby('month')
.agg(avg('precipitation').alias('avg_rainfall'))
.sort('avg_rainfall', ascending=False)
.show()
)

+-----+-------------------+
|month|       avg_rainfall|
+-----+-------------------+
|   11|  5.354166666666667|
|   12|  5.021774193548389|
|    3|  4.888709677419355|
|   10|  4.059677419354839|
|    1| 3.7580645161290316|
|    2|  3.734513274336283|
|    4|  3.128333333333333|
|    9| 1.9624999999999997|
|    5| 1.6733870967741935|
|    8| 1.3201612903225806|
|    6| 1.1075000000000002|
|    7|0.38870967741935486|
+-----+-------------------+



In [52]:
# Which year is the windiest
(
weather.withColumn('year', year('date'))
    .groupby('year')
    .agg(sum('wind').alias('total_wind'))
    .sort(col('total_wind').desc())
    .head(10)
)

[Row(year=2012, total_wind=1244.7),
 Row(year=2014, total_wind=1236.5000000000007),
 Row(year=2015, total_wind=1153.3000000000002),
 Row(year=2013, total_wind=1100.8000000000006)]

In [53]:
(
weather.withColumn('year', year('date'))
    .groupby('year')
    .agg(avg('wind').alias('avg_wind'))
    .sort('avg_wind', ascending=False)
    .show()
)

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2012| 3.400819672131148|
|2014| 3.387671232876714|
|2015| 3.159726027397261|
|2013|3.0158904109589058|
+----+------------------+



In [56]:
# Most frequent type of weather in january
(
weather.withColumn('month', month('date'))
    .filter(col('month') == 1)
    .groupby('weather')
    .count()
    .sort(col('count').desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [57]:
weather.groupby('weather').count().show()

+-------+-----+
|weather|count|
+-------+-----+
|drizzle|   54|
|   rain|  259|
|    sun|  714|
|   snow|   23|
|    fog|  411|
+-------+-----+



In [60]:
# Average high and low temp. on sunny days in july in 2013 and 2014
(
weather.filter(month('date') == 7)
    .filter(year('date') > 2012)
    .filter(year('date') < 2015)
    .filter(col('weather') == lit('sun'))
    .agg(
        avg('temp_max').alias('avg_high_temp'),
        avg('temp_min').alias('avg_low_temp'),
    )
    .show()
)

+-----------------+-----------------+
|    avg_high_temp|     avg_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



In [64]:
weather = weather.withColumn('year', year('date'))
weather = weather.withColumn('month', month('date'))

In [65]:
weather.filter(expr(
    '(year == 2013 OR year == 2014) AND month == 07')).groupby('weather').pivot('year').mean('temp_max').show()
                    

+-------+-----------------+-----------------+
|weather|             2013|             2014|
+-------+-----------------+-----------------+
|    fog|73.34000000000002|           77.792|
|   rain|            71.96|84.91999999999999|
|    sun|79.85333333333334|80.76559999999999|
+-------+-----------------+-----------------+



In [68]:
(
weather.filter(((weather.year == 2013) | (weather.year == 2014)) & (weather.weather == 'sun')).
select(mean('temp_max')).show()
)

+-----------------+
|    avg(temp_max)|
+-----------------+
|65.37874999999998|
+-----------------+



In [66]:
# What percentage of days were rainy in q3 of 2015
# in pandas -- (df.weather == 'rain').mean()
(
weather.filter(year('date') == 2015)
    .filter(quarter('date') == 3)
    .select(when(col('weather') == 'rain', 1).otherwise(0).alias('rain'))
    .agg(mean('rain'))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



In [69]:
weather.show()

+-------------------+-------------+------------------+------------------+----+-------+----+-----+
|               date|precipitation|          temp_max|          temp_min|wind|weather|year|month|
+-------------------+-------------+------------------+------------------+----+-------+----+-----+
|2012-01-01 00:00:00|          0.0|             55.04|              41.0| 4.7|drizzle|2012|    1|
|2012-01-02 00:00:00|         10.9|             51.08|             37.04| 4.5|   rain|2012|    1|
|2012-01-03 00:00:00|          0.8|             53.06|             44.96| 2.3|   rain|2012|    1|
|2012-01-04 00:00:00|         20.3|             53.96|             42.08| 4.7|   rain|2012|    1|
|2012-01-05 00:00:00|          1.3|             48.02|             37.04| 6.1|   rain|2012|    1|
|2012-01-06 00:00:00|          2.5|             39.92|             35.96| 2.2|   rain|2012|    1|
|2012-01-07 00:00:00|          0.0|             44.96|             37.04| 2.3|   rain|2012|    1|
|2012-01-08 00:00:00

In [70]:
weather = weather.withColumn('quarter', quarter('date'))

In [71]:
rain_q3_15 = weather.filter(expr('year == 2015 AND quarter == 3'))

In [72]:
rain_q3_15.where(rain_q3_15.weather == 'rain').count() / rain_q3_15.count()

0.021739130434782608

In [74]:
# For each year, find what percentage of days it rained (non-zero precipitation)
(
weather.withColumn('year', year('date'))
    .select(when(col('precipitation') > 0, 1).otherwise(0).alias('rained'), 'year')
    .groupby('year')
    .agg(mean('rained'))
    .show()
)

+----+-------------------+
|year|        avg(rained)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+



In [75]:
weather = weather.withColumn('rained', when((weather.precipitation > 0), 1).otherwise(0))

In [76]:
weather.groupby('year').agg(mean('rained')).show()

+----+-------------------+
|year|        avg(rained)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+



In [77]:
weather.crosstab('year', 'weather').show()

+------------+-------+---+----+----+---+
|year_weather|drizzle|fog|rain|snow|sun|
+------------+-------+---+----+----+---+
|        2012|     31|  5| 191|  21|118|
|        2013|     16| 82|  60|   2|205|
|        2014|      0|151|   3|   0|211|
|        2015|      7|173|   5|   0|180|
+------------+-------+---+----+----+---+

